# NanoCount command line usage

### Activate virtual environment

In [1]:
conda activate nanocount

(nanocount) 

: 1

### Running NanoCount

In [2]:
NanoCount --help

usage: NanoCount [-h] [--version] -i ALIGNMENT_FILE [-o COUNT_FILE]
                 [-b FILTER_BAM_OUT] [-l MIN_ALIGNMENT_LENGTH]
                 [-f MIN_QUERY_FRACTION_ALIGNED] [-s SEC_SCORING_VALUE]
                 [-t SEC_SCORING_THRESHOLD] [-c CONVERGENCE_TARGET]
                 [-e MAX_EM_ROUNDS] [-x] [-p PRIMARY_SCORE] [-a]
                 [-d MAX_DIST_3_PRIME] [-u MAX_DIST_5_PRIME] [-v] [-q]

NanoCount estimates transcripts abundance from Oxford Nanopore *direct-RNA
sequencing* datasets, using an expectation-maximization approach like RSEM,
Kallisto, salmon, etc to handle the uncertainty of multi-mapping reads

optional arguments:
  -h, --help            show this help message and exit
  --version             show program's version number and exit

Input/Output options:
  -i ALIGNMENT_FILE, --alignment_file ALIGNMENT_FILE
                        BAM or SAM file containing aligned ONT dRNA-Seq reads
                        including secondary and supplementary alignment
    

: 1

#### Basic command

In [6]:
NanoCount -i ./data/aligned_reads_sorted.bam -o ./output/tx_counts.tsv
head ./output/tx_counts.tsv

## Checking options and input files ##
## Initialise Nanocount ##
	Parse Bam file and filter low quality alignments
	Summary of alignments parsed in input bam file
		Valid alignments: 150,517
		Discarded unmapped alignments: 9,545
		Discarded alignment with invalid 3 prime end: 6,133
		Discarded negative strand alignments: 4,515
		Discarded supplementary alignments: 334
	Summary of reads filtered
		Reads with valid best alignment: 85,174
		Invalid secondary alignments: 59,993
		Valid secondary alignments: 2,618
		Reads with low query fraction aligned: 2,362
	Generate initial read/transcript compatibility index
## Start EM abundance estimate ##
	Progress: 2.00 rounds [00:00, 7.82 rounds/s]
	Exit EM loop after 2 rounds
	Convergence value: 0.0019459404177855882
## Summarize data ##
	Convert results to dataframe
	Compute estimated counts and TPM
	Write file
(nanocount) transcript_name	raw	est_count	tpm
YHR174W_mRNA	0.5931738182127508	50522.98679245284	593173.8182127508
YGR192C_mRNA	0.02101

: 1

#### Changing default distance to transcripts ends filters

In [5]:
NanoCount -i ./data/aligned_reads_sorted.bam -o ./output/tx_counts.tsv  --max_dist_3_prime 10 --max_dist_5_prime 10
head ./output/tx_counts.tsv

## Checking options and input files ##
## Initialise Nanocount ##
	Parse Bam file and filter low quality alignments
	Summary of alignments parsed in input bam file
		Valid alignments: 150,517
		Discarded unmapped alignments: 9,545
		Discarded alignment with invalid 3 prime end: 6,133
		Discarded negative strand alignments: 4,515
		Discarded supplementary alignments: 334
	Summary of reads filtered
		Reads with valid best alignment: 85,174
		Invalid secondary alignments: 59,993
		Valid secondary alignments: 2,618
		Reads with low query fraction aligned: 2,362
	Generate initial read/transcript compatibility index
## Start EM abundance estimate ##
	Progress: 2.00 rounds [00:00, 7.56 rounds/s]
	Exit EM loop after 2 rounds
	Convergence value: 0.0019459404177855882
## Summarize data ##
	Convert results to dataframe
	Compute estimated counts and TPM
	Write file
(nanocount) transcript_name	raw	est_count	tpm
YHR174W_mRNA	0.5931738182127508	50522.98679245284	593173.8182127508
YGR192C_mRNA	0.02101

: 1

#### Adding extra transcripts information

The `extra_tx_info` option adds a columns with the transcript lengths and also includes all the zero-coverage transcripts in the results

In [7]:
NanoCount -i ./data/aligned_reads_sorted.bam -o ./output/tx_counts.tsv --extra_tx_info
head ./output/tx_counts.tsv

## Checking options and input files ##
## Initialise Nanocount ##
	Parse Bam file and filter low quality alignments
	Summary of alignments parsed in input bam file
		Valid alignments: 150,517
		Discarded unmapped alignments: 9,545
		Discarded alignment with invalid 3 prime end: 6,133
		Discarded negative strand alignments: 4,515
		Discarded supplementary alignments: 334
	Summary of reads filtered
		Reads with valid best alignment: 85,174
		Invalid secondary alignments: 59,993
		Valid secondary alignments: 2,618
		Reads with low query fraction aligned: 2,362
	Generate initial read/transcript compatibility index
## Start EM abundance estimate ##
	Progress: 2.00 rounds [00:00, 7.76 rounds/s]
	Exit EM loop after 2 rounds
	Convergence value: 0.0019459404177855882
## Summarize data ##
	Convert results to dataframe
	Compute estimated counts and TPM
	Write file
(nanocount) transcript_name	raw	est_count	tpm	transcript_length
YHR174W_mRNA	0.5931738182127508	50522.98679245284	593173.8182127508	13

: 1

#### Write selected alignment to BAM file

In [8]:
NanoCount -i ./data/aligned_reads_sorted.bam -o ./output/tx_counts.tsv -b ./output/aligned_reads_selected.bam --extra_tx_info
head ./output/tx_counts.tsv

## Checking options and input files ##
## Initialise Nanocount ##
	Parse Bam file and filter low quality alignments
	Summary of alignments parsed in input bam file
		Valid alignments: 150,517
		Discarded unmapped alignments: 9,545
		Discarded alignment with invalid 3 prime end: 6,133
		Discarded negative strand alignments: 4,515
		Discarded supplementary alignments: 334
	Summary of reads filtered
		Reads with valid best alignment: 85,174
		Invalid secondary alignments: 59,993
		Valid secondary alignments: 2,618
		Reads with low query fraction aligned: 2,362
	Write selected alignments to BAM file
	Summary of alignments written to bam
		Alignments to select: 87,792
		Alignments written: 87,792
		Alignments skipped: 83,252
	Generate initial read/transcript compatibility index
## Start EM abundance estimate ##
	Progress: 2.00 rounds [00:00, 7.56 rounds/s]
	Exit EM loop after 2 rounds
	Convergence value: 0.0019459404177855882
## Summarize data ##
	Convert results to dataframe
	Compute estim

: 1

#### Relaxing the secondary alignment scoring threshold

The default value is 0.95 (95% of the alignment score of the primary alignment) but this value could be lowered to allow more secondary alignments to be included in the uncertainty calculation.
Lowering the value bellow 0.75 might not be relevant and will considerably increase the computation time.

In [11]:
NanoCount -i ./data/aligned_reads_sorted.bam -o ./output/tx_counts.tsv --sec_scoring_threshold 0.8
head ./output/tx_counts.tsv

## Checking options and input files ##
## Initialise Nanocount ##
	Parse Bam file and filter low quality alignments
	Summary of alignments parsed in input bam file
		Valid alignments: 150,517
		Discarded unmapped alignments: 9,545
		Discarded alignment with invalid 3 prime end: 6,133
		Discarded negative strand alignments: 4,515
		Discarded supplementary alignments: 334
	Summary of reads filtered
		Reads with valid best alignment: 85,174
		Valid secondary alignments: 49,061
		Invalid secondary alignments: 13,550
		Reads with low query fraction aligned: 2,362
	Generate initial read/transcript compatibility index
## Start EM abundance estimate ##
	Progress: 17.0 rounds [00:02, 6.69 rounds/s]
	Exit EM loop after 17 rounds
	Convergence value: 0.004898135236430615
## Summarize data ##
	Convert results to dataframe
	Compute estimated counts and TPM
	Write file
(nanocount) transcript_name	raw	est_count	tpm
YHR174W_mRNA	0.5829682504382943	49653.737762831275	582968.2504382943
YGR192C_mRNA	0.015

: 1

#### verbose mode

Print additional information for QC and debugging

In [13]:
NanoCount -i ./data/aligned_reads_sorted.bam -o ./output/tx_counts.tsv --sec_scoring_threshold 0.8  --verbose

## Checking options and input files ##
	[DEBUG]: Options summary
	[DEBUG]: 	Package name: NanoCount
	[DEBUG]: 	Package version: 0.2.6.post2
	[DEBUG]: 	Timestamp: 2021-08-20 12:24:00.070099
	[DEBUG]: 	alignment_file: ./data/aligned_reads_sorted.bam
	[DEBUG]: 	count_file: ./output/tx_counts.tsv
	[DEBUG]: 	filter_bam_out: 
	[DEBUG]: 	min_alignment_length: 50
	[DEBUG]: 	keep_suplementary: False
	[DEBUG]: 	min_query_fraction_aligned: 0.5
	[DEBUG]: 	sec_scoring_threshold: 0.8
	[DEBUG]: 	sec_scoring_value: alignment_score
	[DEBUG]: 	convergence_target: 0.005
	[DEBUG]: 	max_em_rounds: 100
	[DEBUG]: 	extra_tx_info: False
	[DEBUG]: 	primary_score: primary
	[DEBUG]: 	max_dist_3_prime: 50
	[DEBUG]: 	max_dist_5_prime: -1
	[DEBUG]: 	verbose: True
	[DEBUG]: 	quiet: False
## Initialise Nanocount ##
	Parse Bam file and filter low quality alignments
	Summary of alignments parsed in input bam file
		Valid alignments: 150,517
		Discarded unmapped alignments: 9,545
		Discarded alignment with invalid 3 prim

: 1